<a href="https://colab.research.google.com/github/nvvishnu/CS6910-Assignments/blob/main/Assignment2/partA/Assignment2_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras import *
!pip install wandb
import wandb
from wandb.keras import WandbCallback 

     |████████████████████████████████| 2.1MB 18.3MB/s 
     |████████████████████████████████| 102kB 13.4MB/s 
     |████████████████████████████████| 163kB 44.3MB/s 
     |████████████████████████████████| 133kB 53.9MB/s 
     |████████████████████████████████| 71kB 9.9MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=a726df53009f739538a1214abb48ed771607bf5b9b94f8264cc03c69b19050d6
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=ce9d175d971eba6c2ab8f02f95fcdd64d8e2a320dc107f70dc09cc91668ed7b1
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# # import os
# # path = '/content/drive/MyDrive/nature_12K.zip'
# # import zipfile
# # with zipfile.ZipFile(path, 'rw') as zip_ref:
# #     zip_ref.extractall('/content/drive/MyDrive/Assignment2')
# !pip install split-folders tqdm
# import splitfolders
# splitfolders.ratio(path+'train', output=path+'split', seed=1337, ratio=(.9, .1), group_prefix=None) # default values

In [ ]:
path = '/content/drive/MyDrive/inaturalist_12K/'

In [ ]:
def network(filters,kern_filter,activation_fn,drop_out,batch_normalisation,dense_layer,train,validation,num_classes,epochs,drop_batch_dense,last_2_avgpool,dp_aft_eachCNN):
    
     model = Sequential()
     
     if (batch_normalisation):
        for i in range(5):
          if(i == 0):
            model.add(layers.Conv2D(filters[i],kern_filter[i],input_shape=(224,224,3),activation=activation_fn[i]))     
          else :
            model.add(layers.Conv2D(filters[i],kern_filter[i],activation=activation_fn[i]))

          if (i<=2 or not(last_2_avgpool)):
             if(dp_aft_eachCNN or i>2):
                model.add(layers.Dropout(drop_out))
             model.add(layers.BatchNormalization())
             model.add(layers.MaxPooling2D(pool_size=(2,2)))
          else:
             model.add(layers.Dropout(drop_out))
             model.add(layers.BatchNormalization())
             model.add(layers.AveragePooling2D(pool_size=(2, 2)))
     else:
        for i in range(5):
          if(i == 0):
            model.add(layers.Conv2D(filters[i],kern_filter[i],input_shape=(224,224,3),activation=activation_fn[i]))
          else :
            model.add(layers.Conv2D(filters[i],kern_filter[i],activation=activation_fn[i]))
          if(i<=2 or not(last_2_avgpool)):
             if(i>2 or dp_aft_eachCNN):
               model.add(layers.Dropout(drop_out))
             model.add(layers.MaxPooling2D(pool_size=(2,2)))
          else:
             model.add(layers.Dropout(drop_out))
             model.add(layers.AveragePooling2D(pool_size=(2, 2)))

     model.add(layers.Flatten())
     model.add(layers.Dropout(drop_out))
     model.add(layers.BatchNormalization())
     model.add(layers.Dense(dense_layer,activation='relu'))  #activation_fn[5]))
     if(drop_batch_dense == 'both' or drop_batch_dense == 'drop'):
         model.add(layers.Dropout(drop_out))
     if(drop_batch_dense == 'both' or drop_batch_dense == 'batch'):
         model.add(layers.BatchNormalization())
     model.add(layers.Dense(num_classes))
     model.add(layers.Softmax())
     labels = ['Amphibia','Animalia','Arachnida','Aves','Fungi','Insecta','Mammalia','Mollusca','Plantae','Reptilia']
     print(model.summary())

     model.compile(optimizer='adam',loss=losses.CategoricalCrossentropy(),metrics=['accuracy'])
     model.fit(train,epochs=epochs,validation_data=validation,callbacks=[WandbCallback(validation_data=validation,labels=labels),callbacks.EarlyStopping(patience=10,restore_best_weights=True)])

     return model

In [ ]:
def ImageProcessing(data_augmentation,batch_size):
    
      augmented = preprocessing.image.ImageDataGenerator(
                              #featurewise_center=True,
                              #featurewise_std_normalization=True,
                              rotation_range=20,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              horizontal_flip=True)
      datagen = preprocessing.image.ImageDataGenerator()
      train = []
      validation = []
      if(data_augmentation):
          train = augmented.flow_from_directory(path+'split/train/',target_size=(224,224),color_mode='rgb', classes=None,class_mode='categorical', batch_size=batch_size)
      else:
          train = datagen.flow_from_directory(path+'split/train/',target_size=(224,224),color_mode='rgb', classes=None,class_mode='categorical', batch_size=batch_size)
      validation = datagen.flow_from_directory(path+'split/val/',target_size=(224,224),color_mode='rgb', classes=None,class_mode='categorical', batch_size=batch_size)

      return (train,validation)

In [ ]:
def CNN(filter,filter_organisation,drop_out,batch_normalisation,data_augmentation,kernel_size,batch_size,epochs,drop_batch_dense,last_2_avgpool,dense_sz,dp_aft_eachCNN):

   (train,validation) = ImageProcessing(data_augmentation,batch_size)
   filters = []
   if(filter_organisation == 'same'):
       filters = [filter for i in range(5)]
   if(filter_organisation == 'half'):
       filters = [filter//(2**i) for i in range(5)]
   if(filter_organisation == 'double'):
       filters = [filter*(2**i) for i in range(5)]

   return network(filters,kernel_size,['relu' for i in range(6)],drop_out,batch_normalisation,dense_sz,train,validation,10,epochs,drop_batch_dense,last_2_avgpool,dp_aft_eachCNN)

In [ ]:
def train():
  config_default = { 
        'filter': 16,
        'filter_organisation': 'double',
        'drop_out':0.2,
        'batch_normalisation': True,
        'data_augmentation': True ,
        'kernel_size': [(3,3),(3,3),(5,5),(7,7),(11,11)],
        'batch_size': 32,
        'epochs':10,
        'drop_batch_dense':'both',
        'last_2_avgpool': True,
        'dp_aft_eachCNN' : False,
  }
  print("wand about to be initialised")
  wandb.init(config=config_default,project='Assignment2')
  config = wandb.config

  CNN(config.filter,config.filter_organisation,config.drop_out,config.batch_normalisation,config.data_augmentation,config.kernel_size,config.batch_size,config.epochs,config.drop_batch_dense,config.last_2_avgpool,config.dense_sz,config.dp_aft_eachCNN)
  

In [ ]:
sweep_config = {
    'name': 'CNN',
    'method' : 'bayes',
    'metric': {        
         'goal': 'maximize',
         'name' : 'val_accuracy'
    },
    'parameters': {
        'filter':{
            'values': [32,64]
        },
        'filter_organisation':{
             'values':['double']
        },
        'drop_out':{
             'values':[0.2,0.3]
        },
        'batch_normalisation':{
            'values':[True,False]
        },
        'data_augmentation':{
            'values':[True,False]
        },
        'kernel_size':
        {
            'values':[[(3,3),(3,3),(3,3),(3,3),(3,3)],[(3,3),(3,3),(5,5),(7,7),(7,7)],[(11,11),(5,5),(7,7),(3,3),(3,3)]]
        },
        'batch_size':
        {
            'values':[64]
        },
        'epochs':
        {
            'values':[25]
        },
        'drop_batch_dense':
        {
            'values' : ['both','drop','batch']
        },
        'last_2_avgpool':
        {
            'values':[True,False]
        },
        'dense_sz':
        {
            'values':[128,256,1024]
        },
        'dp_aft_eachCNN':
        {
            'values' : [True,False]
        }
    }
}

In [ ]:
#CNN(256,'same',0.3,True,True,[(3,3),(3,3),(5,5),(7,7),(7,7)],32,15,'drop')

In [ ]:
#sweep_id = wandb.sweep(sweep_config,entity="shivaram_22",project="Assignment2")
sweep_id = '8e4ju7om'

In [ ]:
wandb.agent(sweep_id,project="Assignment2",function=train)

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Agent Starting Run: k2yuqukx with config:
wandb: 	batch_normalisation: True
wandb: 	batch_size: 64
wandb: 	data_augmentation: False
wandb: 	dense_sz: 128
wandb: 	dp_aft_eachCNN: True
wandb: 	drop_batch_dense: both
wandb: 	drop_out: 0.3
wandb: 	epochs: 25
wandb: 	filter: 32
wandb: 	filter_organisation: double
wandb: 	kernel_size: [[3, 3], [3, 3], [3, 3], [3, 3], [3, 3]]
wandb: 	last_2_avgpool: False


wand about to be initialised


wandb: Currently logged in as: shivaram_22 (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Found 8999 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
dropout (Dropout)            (None, 222, 222, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 222, 222, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 64)      18496     
_________________________________________________________________
dropout_1 (Dropout)          (None, 109, 109, 64)      0         
____________________________________

In [ ]:
import os

In [ ]:
path = '/content/drive/MyDrive/inaturalist_12K/split/train/'

In [ ]:
for x in ['Amphibia','Animalia','Arachnida','Aves','Fungi','Insecta','Mammalia','Mollusca','Plantae','Reptilia']:
  cnt = 0
  for file in os.listdir(path+x):
     cnt = cnt + 1
  print(x,' ',cnt)